In [6]:
%matplotlib inline

import json
import numpy as np
import matplotlib.pyplot as plt
import glob
import pandas as pd
from scipy import interpolate
from operator import itemgetter
import matplotlib as mpl
from astropy.stats import sigma_clip
from scipy.stats import iqr

plt.style.use('seaborn-white')
mpl.rcParams.update({'font.size': 16})

## Investigating a Sweep of Window Sizes for Background Fitting

In [7]:
# Select Datasets and extract equivalent widths
outputs = glob.glob("data/ew_known/tame_outputs_windowsize/*.json")
by_hand = glob.glob("data/ew_known/tame_inputs/*.ew")
size_range = np.arange(1.5,5.5,1)
len_range = 5
print(len(by_hand),len(outputs))

d = {'JSON File': outputs, 'By-Hand File': by_hand}
disp_df = pd.DataFrame(data=d)
disp_df

44 44


,By-Hand File,JSON File
0,data/ew_known/tame_inputs/col110_1134red_oned_...,data/ew_known/tame_outputs_windowsize/col110_1...
1,data/ew_known/tame_inputs/col110_2119red_oned_...,data/ew_known/tame_outputs_windowsize/col110_2...
2,data/ew_known/tame_inputs/col110_2129red_oned_...,data/ew_known/tame_outputs_windowsize/col110_2...
3,data/ew_known/tame_inputs/col110_2223red_oned_...,data/ew_known/tame_outputs_windowsize/col110_2...
4,data/ew_known/tame_inputs/col110_3122red_oned_...,data/ew_known/tame_outputs_windowsize/col110_3...
5,data/ew_known/tame_inputs/col110_3144red_oned_...,data/ew_known/tame_outputs_windowsize/col110_3...
6,data/ew_known/tame_inputs/col110_3244red_oned_...,data/ew_known/tame_outputs_windowsize/col110_3...
7,data/ew_known/tame_inputs/col110_4260red_oned_...,data/ew_known/tame_outputs_windowsize/col110_4...
8,data/ew_known/tame_inputs/col110_5125red_oned_...,data/ew_known/tame_outputs_windowsize/col110_5...
9,data/ew_known/tame_inputs/ngc2204_1124red_oned...,data/ew_known/tame_outputs_windowsize/ngc2204_...
